In [1]:
%%bash
pip install sentencepiece
pip install transformers

  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Lib and dataset import

In [2]:
#load libs
from google.colab import files
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import re
import pprint   # For proper print of sequences.
# import treetaggerwrapper
# from googletrans import Translator
from nltk.tokenize import sent_tokenize 
import glob
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
sentence_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/sentence_dataset/sentence_dataset.csv', index_col=0)
sentence_dataset

,chapter_number,page_number,line_number,sentence_number,sentence
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre."
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!
3,1.0,1.0,3,1,Et la lumière fut.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...
...,...,...,...,...,...
40855,66.0,22.0,19,0,et si quelqu'un retranche quelque chose des pa...
40856,66.0,22.0,20,0,"Celui qui atteste ces choses dit: Oui, je vien..."
40857,66.0,22.0,20,1,Amen!
40858,66.0,22.0,20,2,"Viens, Seigneur Jésus!"


# Functions

In [ ]:
# check if GPU is online
import torch
torch.cuda.is_available()

True

In [11]:
from transformers import pipeline
model_checkpoint = "Helsinki-NLP/opus-mt-fr-en"
translator = pipeline("translation", model=model_checkpoint, device=0)


def translate_sentence_faster(input_string, pipe):
  output_string = pipe(input_string)
  return output_string[0]['translation_text']

translate_sentence_faster('ceci est un test', translator)

In [ ]:
tokenizer_fr_en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
model_fr_en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

def translate_sentence(input_string, model, tokenizer):
  input_ids = tokenizer(input_string, return_tensors="pt").input_ids
  outputs = model.generate(input_ids)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)
    

test_dataset = [
  "Et mon cul, c'est du poulet ?",
  " Je vais tomber dans les pommes."
]

for sentence in test_dataset:
  print(sentence + ' -> ' + translate_sentence(sentence, model_fr_en, tokenizer_fr_en))

## Apply translation

In [22]:
# too slow
# sentence_dataset['en_translation'] = sentence_dataset.progress_apply(lambda row: translate_sentence(row['sentence'], model_fr_en, tokenizer_fr_en), axis=1)

# translate EN TO FR
# sentence_dataset['en_fr_translation'] = sentence_dataset.progress_apply(lambda row: translate_sentence(row['en_translation'], model_en_fr, tokenizer_en_fr), axis=1)

sentence_dataset['en_translation'] = sentence_dataset.progress_apply(lambda row: translate_sentence_faster(row['sentence'], translator), axis=1)

sentence_dataset

  0%|          | 0/40860 [00:00<?, ?it/s]

,chapter_number,page_number,line_number,sentence_number,sentence,en_translation
0,1.0,1.0,1,0,"Au commencement, Dieu créa les cieux et la terre.",In the beginning God created the heavens and t...
1,1.0,1.0,2,0,La terre était informe et vide: il y avait des...,The earth was informed and empty: there was da...
2,1.0,1.0,3,0,Dieu dit: Que la lumière soit!,"God said, Let the light be!"
3,1.0,1.0,3,1,Et la lumière fut.,And the light was.
4,1.0,1.0,4,0,Dieu vit que la lumière était bonne; et Dieu s...,God saw that the light was good; and God separ...
...,...,...,...,...,...,...
40855,66.0,22.0,19,0,et si quelqu'un retranche quelque chose des pa...,And if any man cut off anything from the words...
40856,66.0,22.0,20,0,"Celui qui atteste ces choses dit: Oui, je vien...","He who attests to these things said, ""Yes, I a..."
40857,66.0,22.0,20,1,Amen!,Amen!
40858,66.0,22.0,20,2,"Viens, Seigneur Jésus!","Come, Lord Jesus!"


In [23]:
sentence_dataset.to_csv('/content/drive/MyDrive/Colab Notebooks/ressources/la beubli/sentence_dataset/complete_translation.csv')